In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd

In [2]:
class Variable:
    def __init__(self, name, values):
        self.name = name
        self.values = values
        self.indices = { value:idx for idx,value in enumerate(values) }

    def index_of(self, value):
        return self.indices[value]
    
    def __repr__(self):
        return 'Variable: %s, %s' % (self.name, str(self.values))
    def __hash__(self):
        return hash((self.name, tuple(self.values)))
    def __eq__(self, that):
        return self.name == that.name and self.values == that.values

In [3]:
def get_combinations(variables):
        import itertools as it
        var_names = [var.name for var in variables]
        var_values = [var.values for var in reversed(variables)]
        return [list(reversed(tup)) for tup in list(it.product(*var_values))]

class Factor:
    def __init__(self, variables, probas):
        self.variables = variables
        self.probas = probas
    def as_data_frame(self):
        columns = [var.name for var in self.variables]
        columns.append('proba')
        return pd.DataFrame(np.c_[get_combinations(self.variables), 
                                  np.array(self.probas)],columns=columns)

    def assignment_to_index(self, values):
        # values must be in the same order as the variables declared in the constructor
        idx=0
        if (len(values) != len(self.variables)):
            raise Exception('values must have same length as variables')
        total_card=1
        var_idx=0
        for i,var,val in zip(range(len(values)), self.variables, values):
            tmp=var.index_of(val) * total_card
            var_idx += tmp
            total_card *= len(var.values)
        return var_idx
    


In [4]:
def multiply_internal(factor1, factor2, new_vars=None):
    # find common variables
    common_vars = set(factor1.variables).intersection(set(factor2.variables))
    factor1_only_vars = set(factor1.variables).difference(common_vars)
    factor2_only_vars = set(factor2.variables).difference(common_vars)
    #print(common_vars)
    #print(factor1_only_vars)
    #print(factor2_only_vars)
    # create new factor. Have a deterministic way of creating its vars from
    # the factor1 and factor2 vars:
    # - vars of factor1 common with factor2
    # - vars of factor2 common with factor1
    # - vars only in factor1
    # - vars only in factor2
    if (new_vars is None):
        new_vars=[]
        for var in factor1.variables:
            if (var in common_vars):
                new_vars.append(var)
        for var in factor2.variables:
            if (var in common_vars and var not in new_vars):
                new_vars.append(var)
        for var in factor1.variables:
            if (var in factor1_only_vars):
                new_vars.append(var)
        for var in factor2.variables:
            if (var in factor2_only_vars):
                new_vars.append(var)
    else:
        if (set(new_vars) != set( factor1.variables + factor2.variables )):
            raise Exception('invalid new_vars')
    
    new_vars_idx={ var:idx for idx,var in enumerate(new_vars) }
    # print('new_vars_idx: %s' % str(new_vars_idx))
    
    # mapping indices from factor1 to new factor
    fact1_to_new_fact_idx_map={}
    for old_fact_idx,old_fact_var in enumerate(factor1.variables):
        fact1_to_new_fact_idx_map[old_fact_idx] = new_vars_idx[old_fact_var]
    # mapping indices from factor2 to new factor
    fact2_to_new_fact_idx_map={}
    for old_fact_idx,old_fact_var in enumerate(factor2.variables):
        fact2_to_new_fact_idx_map[old_fact_idx] = new_vars_idx[old_fact_var]
        
    # for each row in combs, find assignments in factor 1 and 2 and multiply
    # their respective valuse ==> gives the result for the new factor
    
    import itertools as it
    var_names = [var.name for var in new_vars]
    var_values = [var.values for var in reversed(new_vars)]
    combs=[list(reversed(tup)) for tup in list(it.product(*var_values))]
    # print(combs)

    new_probas=[]
    for idx,row in enumerate(combs):
        # print('row: %s' %str(row))
        fact1_assignment=[ row[fact1_to_new_fact_idx_map[col]] 
                          for col in range(len(factor1.variables)) ]
        fact2_assignment=[ row[fact2_to_new_fact_idx_map[col]] 
                          for col in range(len(factor2.variables)) ]
        # print('fact1_assignment: %s' %str(fact1_assignment))
        # print('fact2_assignment: %s' %str(fact2_assignment))
        
        factor1_proba = factor1.probas[factor1.assignment_to_index(fact1_assignment)]
        factor2_proba = factor2.probas[factor2.assignment_to_index(fact2_assignment)]
        # print('factor1_proba: %s' %str(factor1_proba))
        # print('factor2_proba: %s' %str(factor2_proba))
        new_probas.append(factor1_proba * factor2_proba)
    return Factor(new_vars, new_probas)

def multiply(factors, new_vars=None):
    if (len(factors) < 2):
        raise Exception('must multiply at least 2 factors')
    result_fact = factors[0]
    for i in range(1, len(factors)):
        new_vars_for_mult=None
        if (i == len(factors)-1):
            new_vars_for_mult = new_vars
        result_fact = multiply_internal(result_fact, factors[i], new_vars)
    return result_fact

In [5]:
if (True): # set true for debugging
    i_var = Variable('I', [1,2])
    j_var = Variable('J', [1,2])
    k_var = Variable('K', [1,2])
    ijk_factor = Factor([i_var,j_var,k_var], [1,2,3,4,5,6,7,8])
    ijk_factor.as_data_frame()

In [6]:
if (True): # set true for debugging
    b_var = Variable('B', [1,2])
    kbi_factor = Factor([k_var,b_var,i_var], [1,2,3,4,5,6,7,8])
    kbi_factor.as_data_frame()

In [7]:
if (True): # set true for debugging
    new_factor = multiply([ijk_factor, kbi_factor])
    new_factor.as_data_frame()

In [8]:
def observe_evidence(factor, variable, value):
    variable_col = None
    for i,var in enumerate(factor.variables):
        if (var.name == variable.name):
            variable_col = i
            break
    if (variable_col is None):
        raise Exception('invalid variable ' + str(variable))
    if (value not in variable.values):
        raise Exception('invalid value %s for variable %s' % (str(value), str(variable)))
    new_vars = factor.variables[:variable_col] + factor.variables[variable_col+1:]
    combs = get_combinations(factor.variables)
    new_probas=[]
    for i,comb in enumerate(combs):
        if (comb[variable_col] == value):
            # keep the row for observed evidence, minus the observed column
            new_probas.append(factor.probas[i])
    return Factor(new_vars,new_probas)

In [9]:
if (True): # set true for debugging
    tmp_fact = observe_evidence(new_factor, j_var, 2)
    tmp_fact.as_data_frame()

In [10]:
def remove_var(factor, var_to_remove):
    if (var_to_remove not in factor.variables):
        raise Exception('%s must be in the factor variables' % str(var_to_remove))
    variable_col = None
    for i,var in enumerate(factor.variables):
        if (var.name == var_to_remove.name):
            variable_col = i
            break
    if (variable_col is None):
        raise Exception('invalid variable ' + str(var_to_remove))


    new_vars = list(factor.variables)
    new_vars.remove(var_to_remove)
    combs=get_combinations(new_vars)

    # for each new combination, sum the probas of that combination combined with all
    # values of the removed var
    new_probas=[]
    for comb in combs:
        proba_for_comb = 0
        for value_to_marginalize in var_to_remove.values:
            assignment = comb[:variable_col] + [value_to_marginalize] + comb[variable_col:]
            i = factor.assignment_to_index(assignment)
            proba_for_comb += factor.probas[i]
        new_probas.append(proba_for_comb)

    return Factor(new_vars, new_probas)

def remove_vars(factor, vars_to_remove):
    if (not set(vars_to_remove).issubset(factor.variables)):
        raise Exception('vars_to_remove must be a subset of factor variables')
    if (len(factor.variables) < 2):
        raise Exception('factor must have at least 2 variables')
    result_fact = factor
    for var_to_remove in vars_to_remove:
        result_fact = remove_var(result_fact, var_to_remove)
    return result_fact

In [11]:
def normalize(factor):
    new_probas = list(np.array(factor.probas, dtype='float')/sum(factor.probas))
    return Factor(factor.variables, new_probas)

In [12]:
def copy_factor(factor, new_variables):
    
    if (len(factor.variables) != len(new_variables)):
        raise Exception('new_variables must be of same length as factor.variables')
        
    for i in range(len(new_variables)):
        if (factor.variables[i].values != new_variables[i].values):
            raise Exception('invalid new variables: %s != %s' % 
                            (str(factor.variables[i].values), str(new_variables[i].values)))
        
    return Factor(list(new_variables), list(factor.probas))

def copy_var(variable, new_name):
    return Variable(new_name, list(variable.values))

### Create Bayesian Network
#### Define Variables

In [13]:
genoVarTemplate = Variable('Geno', ['FF','Ff', 'ff'])
phenoVarTemplate = Variable('Pheno', ['CF', 'no CF'])

iraGenoVar = copy_var(genoVarTemplate, 'IraGeno')
robinGenoVar = copy_var(genoVarTemplate, 'RobinGeno')
jamesGenoVar = copy_var(genoVarTemplate, 'JamesGeno')

#### Define Templates
and Factors from Templates

In [14]:
# template for factor giving genotype probability with unknown parents
genoNoParentsFactTemplate = Factor([genoVarTemplate], [0.01, 0.18, 0.81])

robinGenoFact = copy_factor(genoNoParentsFactTemplate, [robinGenoVar])
iraGenoFact = copy_factor(genoNoParentsFactTemplate, [iraGenoVar])
iraGenoFact.as_data_frame()

,IraGeno,proba
0,FF,0.01
1,Ff,0.18
2,ff,0.81


In [15]:
# template for factor giving genotype probability given parents' genotypes
genoGivenParentsGenosTemplate = Factor([genoVarTemplate, genoVarTemplate, genoVarTemplate], 
                               [1, 0, 0, 0.5, 0.5, 0, 0, 1, 0, 0.5, 0.5, 0, 0.25, 0.5, 
                               0.25, 0, 0.5, 0.5, 0, 1, 0, 0, 0.5, 0.5, 0, 0, 1])

jamesGenoGivenParentsFact = copy_factor(genoGivenParentsGenosTemplate, 
                                        [jamesGenoVar, robinGenoVar, iraGenoVar])
jamesGenoGivenParentsFact.as_data_frame()[:5]

,JamesGeno,RobinGeno,IraGeno,proba
0,FF,FF,FF,1.0
1,Ff,FF,FF,0.0
2,ff,FF,FF,0.0
3,FF,Ff,FF,0.5
4,Ff,Ff,FF,0.5


In [16]:
# template for factor giving phenotype probability given parent genotype
phenoGivenGenoFactTemplate = Factor([phenoVarTemplate, genoVarTemplate], [0.8,0.2,0.6,0.4,0.1,0.9])
phenoGivenGenoFactTemplate.as_data_frame()

,Pheno,Geno,proba
0,CF,FF,0.8
1,no CF,FF,0.2
2,CF,Ff,0.6
3,no CF,Ff,0.4
4,CF,ff,0.1
5,no CF,ff,0.9


In [17]:
jamesPhenoVar = copy_var(phenoVarTemplate, 'JamesPheno')
jamesPhenoGivenGenoFact = copy_factor(phenoGivenGenoFactTemplate, 
                                      [jamesPhenoVar, jamesGenoVar])
jamesPhenoGivenGenoFact.as_data_frame()

,JamesPheno,JamesGeno,proba
0,CF,FF,0.8
1,no CF,FF,0.2
2,CF,Ff,0.6
3,no CF,Ff,0.4
4,CF,ff,0.1
5,no CF,ff,0.9


### Compute some Queries
###### First some queries with no evidence (we don't know anything about the actual genotype or phenotype of any person)

In [18]:
# joint distribution of james genotype
jamesGenoJointFact = multiply([jamesGenoGivenParentsFact, robinGenoFact,iraGenoFact],
                             new_vars=[jamesGenoVar,robinGenoVar,iraGenoVar])
jamesGenoJointFact.as_data_frame()

,JamesGeno,RobinGeno,IraGeno,proba
0,FF,FF,FF,0.0001
1,Ff,FF,FF,0.0
2,ff,FF,FF,0.0
3,FF,Ff,FF,0.0009
4,Ff,Ff,FF,0.0009
5,ff,Ff,FF,0.0
6,FF,ff,FF,0.0
7,Ff,ff,FF,0.0081
8,ff,ff,FF,0.0
9,FF,FF,Ff,0.0009


In [19]:
# the probabilities of Jame's genotype without knowing anything about the parents
jamesGenoMarginalFact = normalize(remove_vars(jamesGenoJointFact, [robinGenoVar,iraGenoVar]))
jamesGenoMarginalFact.as_data_frame()

,JamesGeno,proba
0,FF,0.01
1,Ff,0.18
2,ff,0.81


In [20]:
jamesPhenoJointFact=multiply([jamesPhenoGivenGenoFact, jamesGenoJointFact], 
                             new_vars=[jamesPhenoVar, jamesGenoVar, robinGenoVar, iraGenoVar])
jamesPhenoJointFact.as_data_frame()[:8]

,JamesPheno,JamesGeno,RobinGeno,IraGeno,proba
0,CF,FF,FF,FF,8e-05
1,no CF,FF,FF,FF,2e-05
2,CF,Ff,FF,FF,0.0
3,no CF,Ff,FF,FF,0.0
4,CF,ff,FF,FF,0.0
5,no CF,ff,FF,FF,0.0
6,CF,FF,Ff,FF,0.00072
7,no CF,FF,Ff,FF,0.00018


In [21]:
# the probabilities of Jame's phenotype without knowing anything about the parents
jamesPhenoMarginalFact = normalize(remove_vars(jamesPhenoJointFact, [jamesGenoVar,robinGenoVar,iraGenoVar]))
jamesPhenoMarginalFact.as_data_frame()

,JamesPheno,proba
0,CF,0.197
1,no CF,0.803


### Confirmation with Samiam
![genetic](images/genetic.png)

###### Observe that Ira's genotype is FF. Update the probabilities

In [22]:
# recompute the above with evidence that Ira's genotype is FF
jamesGenoJointFact_IraFF = normalize(observe_evidence(jamesGenoJointFact, iraGenoVar, 'FF'))
jamesGenoJointFact_IraFF.as_data_frame()

,JamesGeno,RobinGeno,proba
0,FF,FF,0.01
1,Ff,FF,0.0
2,ff,FF,0.0
3,FF,Ff,0.09
4,Ff,Ff,0.09
5,ff,Ff,0.0
6,FF,ff,0.0
7,Ff,ff,0.81
8,ff,ff,0.0


In [23]:
# James' genotype knowing Ira is FF, knowing nothing about Robin
jamesGenoMarginalFact_IraFF = normalize(remove_vars(jamesGenoJointFact_IraFF, [robinGenoVar]))
jamesGenoMarginalFact_IraFF.as_data_frame()

,JamesGeno,proba
0,FF,0.1
1,Ff,0.9
2,ff,0.0


In [24]:
# James' phenotype joint dist knowing Ira is FF, knowing nothing about Robin
jamesPhenoJointFact_IraFF=multiply([jamesPhenoGivenGenoFact, jamesGenoJointFact_IraFF], 
                             new_vars=[jamesPhenoVar, jamesGenoVar, robinGenoVar])
jamesPhenoJointFact_IraFF.as_data_frame()

,JamesPheno,JamesGeno,RobinGeno,proba
0,CF,FF,FF,0.008
1,no CF,FF,FF,0.002
2,CF,Ff,FF,0.0
3,no CF,Ff,FF,0.0
4,CF,ff,FF,0.0
5,no CF,ff,FF,0.0
6,CF,FF,Ff,0.072
7,no CF,FF,Ff,0.018
8,CF,Ff,Ff,0.054
9,no CF,Ff,Ff,0.036


In [25]:
# James' phenotype knowing Ira is FF, knowing nothing about Robin
jamesPhenoMarginalFact_IraFF = normalize(remove_vars(jamesPhenoJointFact_IraFF, 
                                                     [jamesGenoVar,robinGenoVar]))
jamesPhenoMarginalFact_IraFF.as_data_frame()

,JamesPheno,proba
0,CF,0.62
1,no CF,0.38


### Confirmation with Samiam: Ira FF
![genetic_IraFF](images/genetic_IraFF.png)

###### Observe that Robin's genotype is Ff. Update the probabilities

In [26]:
# if on top of that Robin's genotype is also FF
jamesPhenoJointFact_IraFF_RobinFf = \
   normalize(observe_evidence(jamesPhenoJointFact_IraFF, robinGenoVar, 'Ff'))

jamesPhenoJointFact_IraFF_RobinFf.as_data_frame()

,JamesPheno,JamesGeno,proba
0,CF,FF,0.4
1,no CF,FF,0.1
2,CF,Ff,0.3
3,no CF,Ff,0.2
4,CF,ff,0.0
5,no CF,ff,0.0


In [27]:
jamesPhenoMarginalFact_IraFF_RobinFf = normalize(remove_vars(jamesPhenoJointFact_IraFF_RobinFf, 
                                                     [jamesGenoVar]))
jamesPhenoMarginalFact_IraFF_RobinFf.as_data_frame()

,JamesPheno,proba
0,CF,0.7
1,no CF,0.3


### Confirmation with Samiam: Ira FF, Robin Ff
![genetic_IraFF_robinFf](images/genetic_IraFF_robinFf.png)